<img width="8%" alt="GitHub.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/GitHub.png" style="border-radius: 15%">

# GitHub - Send templates created on a notebooks to Slack channel
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=GitHub+-+Send+templates+created+on+a+notebooks+to+Slack+channel:+Error+short+description">Bug report</a>

**Tags:** #github #templates #created #rest #api #snippet #operations #slack

**Author:** [Benjamin Filly](https://www.linkedin.com/in/benjamin-filly-05427727a/)

**Last update:** 2023-07-11 (Created: 2023-07-11)

**Description:** This notebook demonstrates how to send the templates created on GitHub to a specific Slack channel. It includes the sections below:

- ✅ **Templates created:** the total number of templates created (overall, by month, by week).
- 📊 **Bar chart:** a barchart of the templates created the last 8 weeks

*NB: Execution time may takes between 1 to 4 min.*

**References:**
- [GitHub REST API Documentation](https://docs.github.com/en/rest/issues/issues?apiVersion=2022-11-28#update-an-issue)
- [os](https://docs.python.org/3/library/os.html)
- [Plotly ](https://plotly.com/python/)
- [Slack Block Kit Builder](https://app.slack.com/block-kit-builder/)

## Input

### Import libraries

In [ ]:
from github import Github
import naas
import pandas as pd
from naas_drivers import slack
import plotly.graph_objects as go
from datetime import datetime, timedelta, date
import warnings
warnings.filterwarnings("ignore")

### Setup Variables
- `github_token`: [GitHub token](https://help.github.com/en/github/authenticating-to-github/creating-a-personal-access-token-for-the-command-line)
- `repo_name`: name of the repository in two part: owner_name/repository_name
- `contributor_profile`: your GitHub username
- `cron`: cron params for naas scheduler
- `slack_bot_token`: [Slack Bot token](https://docs.celigo.com/hc/en-us/articles/7140655476507-How-to-create-an-app-and-retrieve-OAuth-token-in-Slack)
- `slack_channel`: The name of the channel you want to send the message

In [ ]:
# Inputs
github_token = naas.secret.get("GITHUB_TOKEN") or "YOUR_GITHUB_TOKEN"
repo_name = "jupyter-naas/awesome-notebooks"
contributor_profile = "Benjifilly" #example: "FlorentLvr" 
cron = "0 17 * * 0"

# Outputs
slack_bot_token = naas.secret.get("SLACK_BOT_TOKEN") or "YOUR_SLACK_TOKEN"
slack_channel = "bot-naas-contributors" #example: "naas-templates-contributors"
image_path = f"bar_chart.png"

## Model

### Connect to GitHub repo

In [ ]:
# Connect to the GitHub API using pygithub library
g = Github(github_token)

# Get the repository
repo = g.get_repo(repo_name)

### Get PRs closed
So here we're filtering out all closed PRs on the `contributor_profile`, then we retrieve the files added by each PR and classify them by month and week.

In [ ]:
# Get the closed PR
pull_requests = repo.get_pulls(state='closed', sort='updated', direction='desc')
print("✅ Pull Requests fetched:", pull_requests.totalCount)

# Iterate over the pull requests and filter by assignee
assigned_pull_requests = [pr for pr in pull_requests if pr.assignee and pr.assignee.login == contributor_profile]

# Print the closed PR count
print(f"📌Number of assigned closed PR: {len(assigned_pull_requests)}")

### Get templates added by PRs closed

In [ ]:
# Create an empty list to store the files added in closed pull requests
files_added_month = []
files_added_week = []
files_added = []

# Get the current date
current_date = datetime.now()

# Iterate over the assigned closed pull requests
for pr in assigned_pull_requests:
    # Get files added
    files = pr.get_files()

    # Get the date the file was created
    file_created = pr.merged_at

    days_diff = None  # Initialize days_diff

    if file_created is not None:
        # Calculate the difference in days between the current date and file creation date
        days_diff = (current_date - file_created).days

        # Iterate over the files added in the pull request
        for file in files:
            if file.status == "added":
                file_path = file.filename
                file_url = file.raw_url

                # Prep data
                tmp = {
                    "file_path": file_path,
                    "file_url": file_url,
                    "file_created": pr.merged_at,
                    "pr_number": pr.number,
                    "pr_title": pr.title,
                    "pr_url": pr.url,
                }

                # Add the data to the respective lists based on the file creation date
                if days_diff is not None and days_diff <= 30:
                    files_added_month.append(tmp)
                if days_diff is not None and days_diff <= 7:
                    files_added_week.append(tmp)

                # Add the data to the files_added list
                files_added.append(tmp)

# Create a DataFrame from the files_added list
df_files_added = pd.DataFrame(files_added)

# Calculate the variation between the current month and the last month
current_month = current_date.month
last_month = current_month - 1 if current_month > 1 else 12
files_added_current_month = df_files_added[df_files_added['file_created'].dt.month == current_month]
files_added_last_month = df_files_added[df_files_added['file_created'].dt.month == last_month]
variation_month = len(files_added_current_month) - len(files_added_last_month)

# Calculate the variation between the current week and the last week
current_week = current_date.isocalendar()[1]
last_week = current_week - 1 if current_week > 1 else 52
files_added_current_week = df_files_added[df_files_added['file_created'].dt.isocalendar().week == current_week]
files_added_last_week = df_files_added[df_files_added['file_created'].dt.isocalendar().week == last_week]
variation_week = len(files_added_current_week) - len(files_added_last_week)

# Prepare the variation strings
variation_month_str = f"+{variation_month}" if variation_month > 0 else str(variation_month)
variation_week_str = f"+{variation_week}" if variation_week > 0 else str(variation_week)

# Print the number of files added and the variations
print(f"📈 Number of Files added: {len(df_files_added)}")
print(f"📁 Number of Files added this month: {len(files_added_current_month)} ({variation_month_str} vs last month)")
print(f"📂 Number of Files added this week: {len(files_added_current_week)} ({variation_week_str} vs last week)")

### Create dataframe and make a graph

In [ ]:
# Prepare data for the bar chart
week_ranges = []  # List to store week ranges
files_added_by_week = []  # List to store the number of files added per week

# Get the current date
current_date = datetime.now().date()
# Get the current week number and year
current_week = datetime.now().strftime("%W")
current_year = datetime.now().strftime("%Y")

# Get the start date of the current week (Monday)
current_start_date = current_date - timedelta(days=current_date.weekday())

# Iterate over the last 8 weeks
for i in range(8):
    start_date = current_start_date - timedelta(weeks=i)
    end_date = start_date + timedelta(days=6)
    week_range = f"{start_date.strftime('%d %b')} - {end_date.strftime('%d %b %Y')}"
    week_ranges.append(week_range)

    # Count the number of files added in the current week
    files_added_in_week = len([file for file in files_added if start_date <= file['file_created'].date() <= end_date])
    files_added_by_week.append(files_added_in_week)
    
# Create the bar chart
fig = go.Figure(
    data=go.Bar(
        x=week_ranges[::-1],
        y=files_added_by_week[::-1],
        text=files_added_by_week[::-1],
        textposition="auto"
    )
)

# Update the layout of the chart
fig.update_layout(
    title= f"Templates created as of W{current_week} {current_year}",
    plot_bgcolor="white",
)

# Get the current timestamp
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")

# Save the chart as an image with the timestamp in the filename
image_path = f"bar_chart_{timestamp}.png"

# Saving the chart into a .png
fig.write_image(image_path)

# Display the chart
fig.show()

### Creating the Slack message
We used [Block kit builder](https://app.slack.com/block-kit-builder/) to create our templates and adapt it with the dynamic variables.

In [ ]:
blocks = [
    {
        "type": "header",
        "text": {
            "type": "plain_text",
            "text": f"{contributor_profile} - Activity update as of {current_date}",
            "emoji": True
        }
    },
    {
        "type": "divider"
    },
    {
        "type": "section",
        "text": {
            "type": "mrkdwn",
            "text": f"✅ *Templates created: {len(files_added)}*",
        }
    },
    {
        "type": "section",
        "text": {
            "type": "mrkdwn",
            "text": f"This month: {len(files_added_current_month)} ({variation_month_str} vs last month)\nThis week: {len(files_added_current_week)} ({variation_week_str} vs last week)"
        }
    },
    {
        "type": "section",
        "text": {
            "type": "mrkdwn",
            "text": "Here are the 5 latest files added this week:\n" + "\n".join([f"- <{file['file_url']}|{file['file_path']}>" for file in files_added_week[:5]])
        }
    },
    {
        "type": "divider"
    },
    {
        "type": "image",
        "title": {
            "type": "plain_text",
            "text": "Evolution by Week"
        },
        "image_url": f"{naas.assets.add(image_path)}",
        "alt_text": "Templates created over the last 8 weeks"
    },
]


## Output

### Send message on Slack

In [ ]:
slack.connect(slack_bot_token).send(slack_channel, text=None, blocks=blocks)

### Add scheduler

In [ ]:
naas.scheduler.add(cron=cron)